# Atividade 07 - SI

 - Marcos Paulo Fontes Feitosa
 - Pedro Hércules de Sousa Dantas
 - José Maria dos Santos Leal

## Importando as bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

## Aquisição dos dados

In [ ]:
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


## Pré-processamento

In [ ]:
X_valid, X_train = X_train[:5000] / 255, X_train[5000:] / 255
y_valid, y_train = y_train[:5000], y_train[5000:]
X_test = X_test / 255

## Construindo e treinando o modelo

In [ ]:
def build_model(n_hidden=1, n_neurons=300, learning_rate=3e-3, input_shape=[28,28]):
  model = keras.models.Sequential()
  model.add(keras.layers.Flatten(input_shape = input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation='relu'))
  model.add(keras.layers.Dense(10, activation='softmax'))
  optimizer = keras.optimizers.SGD(lr=learning_rate)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

In [ ]:
model = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [ ]:
model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/30
1719/1719 [==============================] - 6s 3ms/step - loss: 1.0612 - accuracy: 0.7695 - val_loss: 0.5731 - val_accuracy: 0.8666
Epoch 2/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5017 - accuracy: 0.8770 - val_loss: 0.4193 - val_accuracy: 0.8946
Epoch 3/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4096 - accuracy: 0.8920 - val_loss: 0.3623 - val_accuracy: 0.9052
Epoch 4/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3670 - accuracy: 0.9005 - val_loss: 0.3314 - val_accuracy: 0.9098
Epoch 5/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3405 - accuracy: 0.9058 - val_loss: 0.3107 - val_accuracy: 0.9164
Epoch 6/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3215 - accuracy: 0.9103 - val_loss: 0.2945 - val_accuracy: 0.9188
Epoch 7/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3066 - accuracy: 0.9147 - val_loss: 0.2828 - val_accuracy:

## Otimizando os parâmetros com RandomizedSearchCV

In [ ]:
param_distribs = {
    'n_hidden': [2,3,4],
    'n_neurons': np.arange(268, 400).tolist(),
    'learning_rate': reciprocal(3e-4, 3e-2).rvs(1000).tolist()
}

rnd_searchcv = RandomizedSearchCV(model, param_distribs, n_iter=10, scoring='accuracy', cv=3, verbose=2)

In [ ]:
rnd_searchcv.fit(X_train, y_train, epochs=30, 
                 validation_data = (X_valid, y_valid),
                 callbacks=[keras.callbacks.EarlyStopping(patience=10),
                            keras.callbacks.ModelCheckpoint("melhor_modelo.h5", save_best_only=True)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_neurons=284, n_hidden=4, learning_rate=0.012737650913510548 ...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/30
1146/1146 [==============================] - 8s 6ms/step - loss: 0.7331 - accuracy: 0.7996 - val_loss: 0.3022 - val_accuracy: 0.9140
Epoch 2/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.2800 - accuracy: 0.9173 - val_loss: 0.2121 - val_accuracy: 0.9382
Epoch 3/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.2099 - accuracy: 0.9374 - val_loss: 0.1753 - val_accuracy: 0.9488
Epoch 4/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.1689 - accuracy: 0.9498 - val_loss: 0.1519 - val_accuracy: 0.9560
Epoch 5/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.1398 - accuracy: 0.9584 - val_loss: 0.1335 - val_accuracy: 0.9602
Epoch 6/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.1190 - accuracy: 0.9643 - val_loss: 0.1169 - val_accuracy: 0.9654
Epoch 7/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.1029 - accuracy: 0.9691 - val_loss: 0.1167 - val_accuracy:

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV]  n_neurons=284, n_hidden=4, learning_rate=0.012737650913510548, total= 4.4min
[CV] n_neurons=284, n_hidden=4, learning_rate=0.012737650913510548 ...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.4min remaining:    0.0s


Epoch 1/30
1146/1146 [==============================] - 8s 6ms/step - loss: 0.7023 - accuracy: 0.8078 - val_loss: 0.3000 - val_accuracy: 0.9100
Epoch 2/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.2768 - accuracy: 0.9180 - val_loss: 0.2231 - val_accuracy: 0.9374
Epoch 3/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.2097 - accuracy: 0.9374 - val_loss: 0.1835 - val_accuracy: 0.9474
Epoch 4/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.1674 - accuracy: 0.9512 - val_loss: 0.1530 - val_accuracy: 0.9570
Epoch 5/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.1375 - accuracy: 0.9602 - val_loss: 0.1564 - val_accuracy: 0.9540
Epoch 6/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.1167 - accuracy: 0.9659 - val_loss: 0.1238 - val_accuracy: 0.9642
Epoch 7/30
1146/1146 [==============================] - 7s 6ms/step - loss: 0.1008 - accuracy: 0.9707 - val_loss: 0.1326 - val_accuracy:

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 126.0min finished


1719/1719 [==============================] - 14s 8ms/step - loss: 0.4808 - accuracy: 0.8667 - val_loss: 0.2127 - val_accuracy: 0.9388
Epoch 2/30
1719/1719 [==============================] - 13s 8ms/step - loss: 0.1868 - accuracy: 0.9443 - val_loss: 0.1460 - val_accuracy: 0.9578
Epoch 3/30
1719/1719 [==============================] - 14s 8ms/step - loss: 0.1329 - accuracy: 0.9601 - val_loss: 0.1370 - val_accuracy: 0.9614
Epoch 4/30
1719/1719 [==============================] - 13s 8ms/step - loss: 0.1031 - accuracy: 0.9691 - val_loss: 0.0930 - val_accuracy: 0.9742
Epoch 5/30
1719/1719 [==============================] - 13s 8ms/step - loss: 0.0817 - accuracy: 0.9755 - val_loss: 0.0809 - val_accuracy: 0.9748
Epoch 6/30
1719/1719 [==============================] - 13s 8ms/step - loss: 0.0659 - accuracy: 0.9801 - val_loss: 0.0904 - val_accuracy: 0.9744
Epoch 7/30
1719/1719 [==============================] - 14s 8ms/step - loss: 0.0538 - accuracy: 0.9836 - val_loss: 0.0724 - val_accuracy: 0.9

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f58263cead0>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': [0.00039477699642258534,
                                                          0.0017571390551429425,
                                                          0.0008369238297894399,
                                                          0.004875291747202226,
                                                          0.007057352978801512,
                                                          0.01597572054357195,
                                                          0.006286...
                                                          0.0032856252636772436,
                                                          0.003558922040427927,
                                                          0.0139

## Avaliando o melhor modelo

In [ ]:
best_model = keras.models.load_model('melhor_modelo.h5')


In [ ]:
best_model.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_31 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_122 (Dense)            (None, 344)               270040    
_________________________________________________________________
dense_123 (Dense)            (None, 344)               118680    
_________________________________________________________________
dense_124 (Dense)            (None, 344)               118680    
_________________________________________________________________
dense_125 (Dense)            (None, 344)               118680    
_________________________________________________________________
dense_126 (Dense)            (None, 10)                3450      
Total params: 629,530
Trainable params: 629,530
Non-trainable params: 0
_______________________________________________

In [ ]:
y_pred = np.argmax(best_model.predict(X_test), axis=-1)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score
print('Acurácia: ', accuracy_score(y_test, y_pred))
print('Kappa score: ', cohen_kappa_score(y_test, y_pred))
print('Confusion score: ', confusion_matrix(y_test, y_pred))

Acurácia:  0.9779
Kappa score:  0.975433990788658
Confusion score:  [[ 972    0    0    0    1    1    2    1    2    1]
 [   0 1125    5    0    0    1    2    1    1    0]
 [   1    0 1013    3    2    0    2    9    2    0]
 [   0    1    6  991    0    3    0    6    3    0]
 [   3    0    3    0  964    0    4    2    0    6]
 [   3    1    0   13    2  856   10    2    4    1]
 [   4    2    3    1    6    3  938    0    1    0]
 [   1    2    7    1    0    0    1 1014    1    1]
 [   4    0    7    9    6    3    5    4  933    3]
 [   2    2    0   10   14    0    1    7    0  973]]


## Aplicando a estratégia do funil

In [ ]:
def build_model2(n_hidden=4, n_neurons=344, learning_rate=0.017578226795945062, input_shape=[28,28]):
  model = keras.models.Sequential()
  model.add(keras.layers.Flatten(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation='relu'))
    n_neurons -= (n_neurons * 0.5)
  model.add(keras.layers.Dense(10, activation='softmax'))
  optimizer = keras.optimizers.SGD(lr=learning_rate)
  model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]) 
  return model

In [ ]:
model2 = keras.wrappers.scikit_learn.KerasClassifier(build_model2)

In [ ]:
model2.fit(X_train, y_train, epochs=30,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5097 - accuracy: 0.8553 - val_loss: 0.2031 - val_accuracy: 0.9434
Epoch 2/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1847 - accuracy: 0.9445 - val_loss: 0.1565 - val_accuracy: 0.9556
Epoch 3/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1316 - accuracy: 0.9606 - val_loss: 0.1096 - val_accuracy: 0.9678
Epoch 4/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.1027 - accuracy: 0.9690 - val_loss: 0.1027 - val_accuracy: 0.9710
Epoch 5/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.0818 - accuracy: 0.9758 - val_loss: 0.0867 - val_accuracy: 0.9744
Epoch 6/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.0669 - accuracy: 0.9810 - val_loss: 0.0894 - val_accuracy: 0.9750
Epoch 7/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.0557 - accuracy: 0.9835 - val_loss: 0.0736 - val_accuracy:

In [ ]:
y_pred = np.argmax(model2.predict_proba(X_test), axis=-1)

In [ ]:
print('Acurácia: ', accuracy_score(y_test, y_pred))
print('Kappa score: ', cohen_kappa_score(y_test, y_pred))
print('Confusion score: ', confusion_matrix(y_test, y_pred))

Acurácia:  0.9801
Kappa score:  0.9778803757828485
Confusion score:  [[ 967    0    0    0    1    3    3    1    3    2]
 [   0 1126    1    2    0    2    2    1    1    0]
 [   2    0 1014    5    2    0    2    3    4    0]
 [   0    0    4  990    0    4    0    2    5    5]
 [   0    0    3    0  962    0    4    3    0   10]
 [   2    0    0    7    0  874    3    0    4    2]
 [   5    2    0    0    7    7  937    0    0    0]
 [   1    5    6    3    1    1    0 1002    3    6]
 [   5    0    1    5    6    4    2    4  942    5]
 [   2    2    0    7    5    1    2    1    2  987]]


In [ ]:
model2.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 344)               270040    
_________________________________________________________________
dense_1 (Dense)              (None, 172)               59340     
_________________________________________________________________
dense_2 (Dense)              (None, 86)                14878     
_________________________________________________________________
dense_3 (Dense)              (None, 43)                3741      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                440       
Total params: 348,439
Trainable params: 348,439
Non-trainable params: 0
__________________________________________________